In [ ]:
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.layers as layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from tools import create_split_gen, plot_fitting_history,class_report,find_errors
tensorflow.keras.utils.set_random_seed(812)
tensorflow.config.experimental.enable_op_determinism()

In [ ]:
train_augmentation = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,   
    height_shift_range=0.1 )
train_gen = create_split_gen(train_augmentation, path="/home/ocelot/Desktop/CNN_RPS/Archive/train", label_type="sparse")

val_gen = create_split_gen(ImageDataGenerator(rescale = 1./255), path="/home/ocelot/Desktop/CNN_RPS/Archive/validation", label_type="sparse")
    

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
def crea_modello(drop_param = 0.5, learning_rate=0.001, num_filter_1=32, num_filter_2=64):
    model = tensorflow.keras.models.Sequential([
        layer.InputLayer(shape=(100,150,1)),
        layer.Conv2D(filters=(num_filter_1), kernel_size=(3,3), activation="relu"),
        layer.MaxPooling2D(pool_size=(2,2)),
        layer.Conv2D(filters=(num_filter_2), kernel_size=(3,3), activation="relu"),
        layer.MaxPooling2D(pool_size=(2,2)),
        layer.Flatten(),
        layer.Dense(128, activation="relu"),
        layer.Dropout(drop_param),
        layer.Dense(3, activation="softmax")
    ])
    optimizer = tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model
    

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
model = KerasClassifier(model = crea_modello, epochs=15, batch_size=32,verbose=0)
griglia = {
    "model__drop_param": [0.25, 0.5],
    "model__learning_rate":[1e-3, 5e-4, 1e-4],
    "model__num_filter_1":[32,64],
    "model__num_filter_2":[32,64]
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=griglia,
    cv=kfold,               
    scoring='accuracy',         
    verbose=0,
    n_jobs=1 )

In [ ]:
import os
import cv2
Im = []
target = []

cat = ["paper", "scissors", "rock"]
vettori = {"paper":0, "scissors":2, "rock":1}
for i in cat:
    for j in os.listdir(os.path.join("/home/ocelot/Desktop/CNN_RPS/Archive/train", i)):
        Im.append(cv2.imread(os.path.join("/home/ocelot/Desktop/CNN_RPS/Archive/train", i,j), 0))
        target.append(vettori[i])
X_train = np.array(Im).astype('float32') / 255.0
X_train = np.expand_dims(X_train, axis=-1)
Y_train = np.array(target)

In [ ]:
%%time
risultati = grid_search.fit(X_train, Y_train )

In [ ]:
print("Best parameters:", grid_search.best_params_)

In [ ]:
final_model = crea_modello(drop_param=0.25, learning_rate=0.0005, num_filter_1=64, num_filter_2=32)
history= final_model.fit(train_gen, epochs=50, callbacks=[early_stop], validation_data=val_gen)

In [ ]:
plot_fitting_history(history)

In [ ]:
test_gen = create_split_gen(ImageDataGenerator(rescale = 1./255), path= "/home/ocelot/Desktop/CNN_RPS/Archive/test", shuffle=False, label_type="sparse")

In [ ]:
testLoss, testAccuracy = final_model.evaluate(test_gen)

In [ ]:
pred = final_model.predict(test_gen)
report = class_report(pred, test_gen)

In [ ]:
report["scissors"]

In [ ]:
report["rock"]

In [ ]:
report["paper"]

In [ ]:
find_errors(pred, test_gen.classes, test_gen)

In [ ]:
final_model.save("/home/ocelot/Desktop/secondo.keras")